In [ ]:
# A warning message from DGL will appear which seems to be related to an open issue in the DGL library, 
# it won't hurt the execution of the program, please ignore it.
import quartz

In [ ]:
quartz_context = quartz.QuartzContext(gate_set=['ccz', 'h', 'cx', 'rz', 'x', 'add'], filename='../Nam_5_3_complete_ECC_set.json')
parser = quartz.PyQASMParser(context=quartz_context)
my_dag = parser.load_qasm(filename="../circuit/nam-benchmarks/barenco_tof_3.qasm")
init_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)
init_graph = init_graph.ccz_flip_greedy_rz()

The codes below shows the usage of the two APIs `PyGraph.available_xfers` and `PyGraph.apply_xfer`.

In [ ]:
# available_xfer_matrix = init_graph.get_available_xfers_matrix(context=quartz_context)
# all_nodes = init_graph.all_nodes()
# for node in all_nodes:
#     print(init_graph.available_xfers(context=quartz_context, node=node))

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

def check(graph):
    graph.to_qasm(filename='best.qasm')
    qc_origin = QuantumCircuit.from_qasm_file('barenco_tof_3_opt_path/subst_history_39.qasm')
    qc_optimized = QuantumCircuit.from_qasm_file('best.qasm')
    return Statevector.from_instruction(qc_origin).equiv(Statevector.from_instruction(qc_optimized))

In [ ]:
# Optimizing with BFS
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy

candidate_hq = []
heapq.heappush(candidate_hq, init_graph)
hash_set = set()
hash_set.add(init_graph.hash())
best_graph = init_graph
best_gate_cnt = init_graph.gate_count

budget = 5_000_000

while candidate_hq != [] and budget >= 0:
    first_candidate = heapq.heappop(candidate_hq)
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=32) as executor:
        results = executor.map(ax, list(range(len(all_nodes))), chunksize=2)
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            new_hash = new_graph.hash()
            if new_hash not in hash_set:
                hash_set.add(new_hash)
                heapq.heappush(candidate_hq, new_graph)
                new_cnt = new_graph.gate_count
                if new_cnt < best_gate_cnt:
                    best_graph = new_graph
                    best_gate_cnt = new_cnt
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}')
